# Computing sentence embeddings

A first prototype on a small corpus, following https://openreview.net/pdf?id=SyK00v5xx.

The basic idea is to compute the sentence vector v_s as a weighted average of the word vectors associated with the words contained in s. The weight of each vector is an inverse function of the probability of the corresponding word in the corpus:

![s_formula](img/algorithm.png)

We need the following ingredients:

1) a set of sentences (corpus)

2) the vocabulary of corpus (unique tokens)

3) the probability of each word in the vocabulary (frequency in the corpus)

4) a word vector (embedding) for each word

Import basic packages first:

In [1]:
# basic
import numpy as np
import re # regular expressions

The parameter a is manually set to 0.001 (see paper):

In [2]:
a = 0.001

Next, we load our corpus, which is a random sample of 50k sentences from OpenSubtitles corpus available at http://opus.nlpl.eu/OpenSubtitles-v2018.php. The full corpus has been slightly cleaned and the sample obtained with bash command shuf.

In [3]:
with open('data/mini-subtitles-corpus', 'r') as input_file:
    corpus = input_file.read()

corpus = re.sub('\n', '. ', corpus) # we'll split sentences based on full stops with spacy

In [4]:
print(len(corpus))
corpus[0:1000]

2168174


"ok, beh, apprezzo la tua preoccupazione. avresti dovuto dirle prima queste cose. ti stai divertendo, blaine. sono miei ormai. quando ero a princeton, ho scritto la mia tesi sulle passioni degli stoici. hai una macchia sul tuo file, delinquente. presto anche tu capirai. se non lo fai, lo chiamo e gli dico dove sei. beh, sarebbe stato prima che venissi uccisa, ma funziona anche così. usa la presa al ginocchio. dolore e dai desideri. di che puntualizzazione parli. nella mia fantasia, noi ci trasferiamo nell'attico di jennifer lawrence. mazzola. perché loro non si lavano. non se ne adrà via. ah, 50ooo lire, don pietro. bob e malcolm sono stati licenziati. seq druven begnan si. molti amici mi hanno chiesto di farti ripensare a questa cosa. avrei dovuto lasciarla morire per evitare che lei odiasse me. erano un bersaglio facile mentre stavano dormendo nei loro nidi, di giorno. qual è stato il primo film western a vincere l'oscar come miglior film. magari mi fai vedere l'insegnante di inglese

A bunch of sentences, as expected.

Next, we parse the corpus with spacy (it'll take a while):

In [5]:
# import spaCy for nlp and italian resources (install if necessary)

#!pip3 install spacy
#!python3 -m spacy download it

import spacy
nlp = spacy.load('it')

In [6]:
nlp.max_length = 2500000
doc = nlp(corpus)

In [7]:
sentences = [sentence for sentence in doc.sents]

In [8]:
sentences[0:9]

[ok, beh, apprezzo la tua preoccupazione.,
 avresti dovuto dirle prima queste cose.,
 ti stai divertendo, blaine.,
 sono miei ormai.,
 quando ero a princeton, ho scritto la mia tesi sulle passioni degli stoici.,
 hai una macchia sul tuo file, delinquente.,
 presto anche tu capirai.,
 se non lo fai, lo chiamo e gli dico dove sei.,
 beh, sarebbe stato prima che venissi uccisa, ma funziona anche così.]

In [9]:
len(sentences)

49669

Get tokens, stripping punctuation:

In [10]:
tokens = [token.text for token in doc if token.is_punct != True]

In [11]:
len(tokens)

375400

Get frequency of each token:

In [12]:
# we use Counter from collections package
from collections import Counter

In [13]:
tokens_count = Counter(tokens)

For example:

In [14]:
tokens_count.most_common(10)

[('di', 9946),
 ('che', 9785),
 ('non', 8535),
 ('è', 8055),
 ('e', 6578),
 ('la', 6408),
 ('il', 6134),
 ('un', 5647),
 ('a', 5643),
 ('per', 4860)]

In [15]:
tokens_count['cane']

43

Get list of unique tokens:

In [16]:
unique_tokens = set(tokens)

In [17]:
vocab_size = len(unique_tokens)
print(vocab_size)

34870


With this list we can put together a dictionary of unique tokens with their probability in the corpus:

In [18]:
# iterating on the keys of tokens_count object, we divide the count of each token by the length of the vocabulary
tokens_prob = {key : tokens_count[key]/vocab_size for key in tokens_count.keys()}

For example:

In [19]:
tokens_prob['il']

0.17591052480642386

In [20]:
tokens_prob['cane']

0.0012331517063378262

In [21]:
tokens_prob['segugio']

2.8677946659019214e-05

Next, we train Word2Vec model on our corpus with gensim:

In [22]:
# gensim is used to load word embeddings (install if necessary)

#!pip3 install gensim

from gensim.models import Word2Vec

We need tokenized sentences as input for Word2Vec:

In [23]:
# double list comprehension: collect tokens, stripping punctuation, for each sentence in doc
tokenized_sentences = [[token.text for token in sentence if token.is_punct != True] for sentence in sentences]

For example:

In [24]:
tokenized_sentences[0:9]

[['ok', 'beh', 'apprezzo', 'la', 'tua', 'preoccupazione'],
 ['avresti', 'dovuto', 'dirle', 'prima', 'queste', 'cose'],
 ['ti', 'stai', 'divertendo', 'blaine'],
 ['sono', 'miei', 'ormai'],
 ['quando',
  'ero',
  'a',
  'princeton',
  'ho',
  'scritto',
  'la',
  'mia',
  'tesi',
  'sulle',
  'passioni',
  'degli',
  'stoici'],
 ['hai', 'una', 'macchia', 'sul', 'tuo', 'file', 'delinquente'],
 ['presto', 'anche', 'tu', 'capirai'],
 ['se', 'non', 'lo', 'fai', 'lo', 'chiamo', 'e', 'gli', 'dico', 'dove', 'sei'],
 ['beh',
  'sarebbe',
  'stato',
  'prima',
  'che',
  'venissi',
  'uccisa',
  'ma',
  'funziona',
  'anche',
  'così']]

In [25]:
vec_size = int(vocab_size ** 0.25) # rule of thumb to decide size of embedding vectors
model = Word2Vec(tokenized_sentences,size=vec_size, window=5, min_count=1, workers=4)

For example:

In [26]:
model.wv['cane'][0:9] # show only the first nine values

array([ 0.04793363,  0.14515889, -0.5201173 , -0.41402858,  0.2684565 ,
       -0.255092  ,  0.3482258 , -0.810052  ,  0.33293155], dtype=float32)

In [27]:
model.wv['lupo'][0:9] # show only the first nine values

array([ 0.05598346,  0.21135338, -0.28277728, -0.20828103,  0.20839186,
       -0.15002275,  0.1731048 , -0.3455772 ,  0.20046158], dtype=float32)

In [28]:
model.wv['gatto'][0:9] # show only the first nine values

array([-0.06719015,  0.11652127, -0.20855701, -0.20998985,  0.14426757,
       -0.0671446 ,  0.09711676, -0.3021662 ,  0.20518269], dtype=float32)

Double check vocabulary:

In [29]:
model_unique_tokens = set([token for token in model.wv.vocab]) # unique tokens in model vocab
model_unique_tokens == set(unique_tokens) # exaclty the same as unique_tokens above?

True

Cool.

We have all our ingredients: sentences, tokens, probabilities and vectors.

Let's move to sentence embedding algorithm:

In [30]:
def compute_s_vec(sentence, a=0.001): # make sure sentence is tokenized! 

    sent_vec = np.zeros(shape=vec_size) # initialize vector of zeros with the wanted shape

    for token in sentence: # cycle through tokens in sentence
        token_p = tokens_prob[token] # probability of token
        token_vec = model.wv[token] # token vector
        weighted_token_vec = token_vec*(a/(a+token_p)) # weighted vector of token
        sent_vec = sent_vec + weighted_token_vec # sum

    sent_vec = sent_vec*(1/len(sent_vec)) # average

    return(sent_vec)

For example:

In [31]:
il_cane_lupo = compute_s_vec(['il', 'cane', 'lupo'])
il_cane_lupo[0:9]

array([ 0.00506044,  0.01638964, -0.03351006, -0.02523877,  0.01940483,
       -0.01551588,  0.02173505, -0.04574097,  0.02202952])

In [32]:
il_cane_gatto = compute_s_vec(['il', 'cane', 'gatto'])
il_cane_gatto[0:9]

array([-0.00121915,  0.01220169, -0.03068349, -0.02612948,  0.01682468,
       -0.0117225 ,  0.0183937 , -0.04478033,  0.02304901])

Finally, create a dictionary computing vector for each sentence in corpus:

In [33]:
sentences_vector = {sentences[i] : compute_s_vec(tokenized_sentences[i]) for i in range(len(sentences))}

To do: steps 4-7 of the algorithm above.